<a href="https://colab.research.google.com/github/minhhieu9800/AIN501/blob/main/MSEK10_K11_NguyenMinhHieu_AIN501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Kết nối đến GitHub
!git clone https://github.com/minhhieu9800/AIN501.git

Cloning into 'AIN501'...
remote: Enumerating objects: 3463, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 3463 (delta 0), reused 125 (delta 0), pack-reused 3270
Receiving objects: 100% (3463/3463), 83.15 MiB | 25.55 MiB/s, done.


In [37]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
import keras.callbacks as callbacks
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

import requests
from PIL import Image
from io import BytesIO

In [4]:
# Khai báo đường dẫn cho tập dữ liệu train và validate
train_dir = '/content/AIN501/train'
val_dir = '/content/AIN501/validate'

# Thông số cho tập dữ liệu
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32
epochs = 50
num_classes = len([f for f in os.listdir(train_dir) if not f.endswith('.zip')])

print(num_classes)

# Tạo data generator cho tập train và validate
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Xây dựng mô hình CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

# Lưu model
model.save('logo_model.h5')

# Đánh giá mô hình
score = model.evaluate(val_generator, val_generator.samples // batch_size, workers=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


44
Found 1213 images belonging to 44 classes.
Found 1609 images belonging to 44 classes.
Epoch 1/50
37/37 [==============================] - 22s 583ms/step - loss: 3.9649 - accuracy: 0.0322 - val_loss: 3.7850 - val_accuracy: 0.0244
Epoch 2/50
37/37 [==============================] - 21s 576ms/step - loss: 3.7634 - accuracy: 0.0390 - val_loss: 3.7796 - val_accuracy: 0.0388
Epoch 3/50
37/37 [==============================] - 21s 568ms/step - loss: 3.7321 - accuracy: 0.0644 - val_loss: 3.7586 - val_accuracy: 0.0463
Epoch 4/50
37/37 [==============================] - 21s 563ms/step - loss: 3.6800 - accuracy: 0.0787 - val_loss: 3.7274 - val_accuracy: 0.0444
Epoch 5/50
37/37 [==============================] - 20s 548ms/step - loss: 3.6288 - accuracy: 0.0847 - val_loss: 3.6468 - val_accuracy: 0.0688
Epoch 6/50
37/37 [==============================] - 22s 606ms/step - loss: 3.5039 - accuracy: 0.1245 - val_loss: 3.5622 - val_accuracy: 0.1031
Epoch 7/50
37/37 [==============================] - 2

<ipython-input-4-99c40bb96463>:68: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(val_generator, val_generator.samples // batch_size, workers=1)


Test loss: 1.854241967201233
Test accuracy: 0.6381250023841858


In [63]:
def predict_logo(image_path):
  class_names = sorted(os.listdir(train_dir))
  model = load_model("logo_model.h5")
  # đường dẫn đến file ảnh logo cần dự đoán
  # img_path = image_path

  url = image_path
  response = requests.get(url)

  # mở ảnh từ response
  img = Image.open(BytesIO(response.content))

  # Chuyển ảnh về RGB
  img = img.convert('RGB')

  img = img.resize((224, 224))

  img_array = np.array(img)
  x = np.expand_dims(img_array, axis=0)
  x = x / 255.0

  # dự đoán
  preds = model.predict(x)

  # print(class_names)
  # print(preds)
  # print(model.classes) 

  # lấy chỉ số của lớp có xác suất dự đoán cao nhất
  class_idx = np.argmax(preds[0])

  # print(class_idx)

  # in ra tên của hãng xe
  print(class_names[class_idx])


In [65]:
predict_logo("https://1000logos.net/wp-content/uploads/2018/03/Honda-logo.jpg")

1/1 [==============================] - 0s 94ms/step
HONDA
